In [31]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Polygon, Point

In [27]:
raw_df = pd.read_csv('data\OBIS_anemone_occurrences_slim.tsv', sep='\t')

In [28]:
raw_df['geometry'] = raw_df['geometry'].apply(wkt.loads)

raw_gdf = gpd.GeoDataFrame(raw_df, geometry='geometry', crs={'init': 'epsg:4326'})

c:\Users\geo1k\anaconda3\envs\geospark\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [46]:
def create_grid(xmin, xmax, ymin, ymax, grid_size):
    # Create lists of edges for the grid
    cols = list(range(int(xmin), int(xmax) + 1, grid_size))
    rows = list(range(int(ymin), int(ymax) + 1, grid_size))
    rows.reverse()

    # Create polygons for each cell in the grid
    polygons = []
    tile_ids = []
    for x in cols[:-1]:
        for y in rows[:-1]:
            polygons.append(Polygon([(x, y), (x + grid_size, y), (x + grid_size, y + grid_size), (x, y + grid_size)]))
            lat_id = f"{abs(y)}{'N' if y >= 0 else 'S'}"
            lon_id = f"{abs(y)}{'E' if x >= 0 else 'W'}"
            tile_ids.append(f"{lat_id}_{lon_id}")

    # Create a GeoDataFrame
    grid = gpd.GeoDataFrame({'TileID': tile_ids, 'geometry': polygons}, crs={'init': 'epsg:4326'})
    return grid

# Specify the bounds of the grid (-180 to 180, -90 to 90) and the size of each cell (10x10 degrees)
grid = create_grid(-180, 180, -90, 90, 10)


c:\Users\geo1k\anaconda3\envs\geospark\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\geo1k\anaconda3\envs\geospark\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [47]:
grid

,TileID,geometry
0,90N_90W,"POLYGON ((-180.00000 90.00000, -170.00000 90.0..."
1,80N_80W,"POLYGON ((-180.00000 80.00000, -170.00000 80.0..."
2,70N_70W,"POLYGON ((-180.00000 70.00000, -170.00000 70.0..."
3,60N_60W,"POLYGON ((-180.00000 60.00000, -170.00000 60.0..."
4,50N_50W,"POLYGON ((-180.00000 50.00000, -170.00000 50.0..."
...,...,...
643,40S_40E,"POLYGON ((170.00000 -40.00000, 180.00000 -40.0..."
644,50S_50E,"POLYGON ((170.00000 -50.00000, 180.00000 -50.0..."
645,60S_60E,"POLYGON ((170.00000 -60.00000, 180.00000 -60.0..."
646,70S_70E,"POLYGON ((170.00000 -70.00000, 180.00000 -70.0..."


In [48]:
joined = gpd.sjoin(grid, raw_gdf, how="left", op='contains')

c:\Users\geo1k\anaconda3\envs\geospark\Lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [49]:
grid['Count'] = joined.groupby('TileID').size().reindex(grid['TileID'], fill_value=0)

ValueError: cannot reindex on an axis with duplicate labels